In [1]:
import sys
sys.path.append('../')

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2,3"
import torch
import pandas as pd

from utils.utils_data import get_camelyon_test_dataloader, get_wssb_test_dataloader
from models.DVBCDModel import DVBCDModel

print(torch.__version__)
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
torch.autograd.set_detect_anomaly(True)

USE_GPU = True
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
if USE_GPU and torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print('using device:', DEVICE)

1.12.1
using device: cuda


In [2]:
MAIN_PATH = "/work/work_fran/Deep_Var_BCD/"
SAVE_MODEL_PATH = MAIN_PATH + "weights/0pe_224ps_0.25theta_0.05sigmaRui_60000nsamples/"

N_SAMPLES = 60000
PATCH_SIZE = 224
NUM_WORKERS = 64
WSSB_DATA_PATH = "/data/BasesDeDatos/Alsubaie/Data/"
CAMELYON_DATA_PATH = "/data/BasesDeDatos/Camelyon/Camelyon17/training/patches_224/"

In [3]:
model = DVBCDModel(device=DEVICE)
model.load(SAVE_MODEL_PATH + "best.pt", remove_module=False)

In [4]:
test_dataloader_wssb_dict = get_wssb_test_dataloader(WSSB_DATA_PATH, NUM_WORKERS)
results_dic_wssb = {}
for organ in test_dataloader_wssb_dict.keys():
    metrics_wssb = model.evaluate_GT(test_dataloader_wssb_dict[organ])
    for k in metrics_wssb.keys():
        results_dic_wssb[f"wssb_{organ}_" + k] = metrics_wssb[k]
res_wssb_df = pd.DataFrame(results_dic_wssb, index=[0])
res_wssb_df

Testing: 100%|██████████| 14/14 [00:01<00:00,  7.81it/s, test_loss=0.39291, test_loss_mse=0.00014, test_loss_kl=0.39291, test_mse_rec=0.00025, test_psnr_rec=26.52363, test_ssim_rec=0.96605, test_mse_gt_h=0.11897, test_mse_gt_e=0.0326, test_psnr_gt_h=18.86466, test_psnr_gt_e=19.73756, test_ssim_gt_h=0.89368, test_ssim_gt_e=0.57814]


,wssb_Lung_test_loss,wssb_Lung_test_loss_mse,wssb_Lung_test_loss_kl,wssb_Lung_test_mse_rec,wssb_Lung_test_psnr_rec,wssb_Lung_test_ssim_rec,wssb_Lung_test_mse_gt_h,wssb_Lung_test_mse_gt_e,wssb_Lung_test_psnr_gt_h,wssb_Lung_test_psnr_gt_e,...,wssb_Colon_test_loss_kl,wssb_Colon_test_mse_rec,wssb_Colon_test_psnr_rec,wssb_Colon_test_ssim_rec,wssb_Colon_test_mse_gt_h,wssb_Colon_test_mse_gt_e,wssb_Colon_test_psnr_gt_h,wssb_Colon_test_psnr_gt_e,wssb_Colon_test_ssim_gt_h,wssb_Colon_test_ssim_gt_e
0,1.625871,0.000127,1.625871,0.000156,27.974915,0.964691,0.069712,0.018176,23.178846,23.564484,...,0.392906,0.000253,26.523631,0.96605,0.118965,0.032595,18.86466,19.737556,0.893677,0.578136


In [6]:
res_wssb_df.transpose()

,0
wssb_Lung_test_loss,1.625871
wssb_Lung_test_loss_mse,0.000127
wssb_Lung_test_loss_kl,1.625871
wssb_Lung_test_mse_rec,0.000156
wssb_Lung_test_psnr_rec,27.974915
wssb_Lung_test_ssim_rec,0.964691
wssb_Lung_test_mse_gt_h,0.069712
wssb_Lung_test_mse_gt_e,0.018176
wssb_Lung_test_psnr_gt_h,23.178846
wssb_Lung_test_psnr_gt_e,23.564484


In [5]:
#n_samples_camelyon = int(0.2*N_SAMPLES)
#test_dataloader_camelyon = get_camelyon_test_dataloader(CAMELYON_DATA_PATH, PATCH_SIZE, NUM_WORKERS, n_samples_camelyon)
#results_dic_camelyon = {}
#metrics_camelyon = model.evaluate(test_dataloader_camelyon)
#for k in metrics_camelyon.keys():
#    results_dic_camelyon["camelyon_" + k] = metrics_camelyon[k]
#res_camelyon_df = pd.DataFrame(results_dic_camelyon, index=[0])
#res_camelyon_df